# Using EMI-RNN on the HAR Dataset

This is a very simple example of how the existing EMI-RNN implementation can be used on the HAR dataset. We illustrate how to train a model that predicts on 48 step sequence in place of the 128 length baselines while attempting to predict early. For more advanced use cases which involves more sophisticated computation graphs or loss functions, please refer to the doc strings provided with the released code.

In the preprint of our work, we use the terms *bag* and *instance* to refer to the LSTM input sequence of original length and the shorter ones we want to learn to predict on, respectively. In the code though, *bag* is replaced with *instance* and *instance* is replaced with *sub-instance*. To avoid ambiguity, we will use the terms *bag* and *sub-instance*  throughout this document.

The network used here is a simple LSTM + Linear classifier network. 

The UCI [Human Activity Recognition](https://archive.ics.uci.edu/ml/datasets/human+activity+recognition+using+smartphones) dataset.

In [1]:
import os
import sys
import tensorflow as tf
import numpy as np
sys.path.insert(0, '../')
os.environ['CUDA_VISIBLE_DEVICES'] =''

# MI-RNN and EMI-RNN imports
from edgeml.graph.rnn import EMI_DataPipeline
from edgeml.graph.rnn import EMI_BasicLSTM
from edgeml.trainer.emirnnTrainer import EMI_Trainer, EMI_Driver
import edgeml.utils

Let us set up some network parameters for the computation graph. These will be explained later.

In [2]:
# Network parameters for our LSTM + FC Layer
NUM_HIDDEN = 16
NUM_TIMESTEPS = 48
NUM_FEATS = 9
FORGET_BIAS = 1.0
NUM_OUTPUT = 6
USE_DROPOUT = False
KEEP_PROB = 0.9

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 32

# Number of epochs in *one iteration*
NUM_EPOCHS = 5
# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 3
# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 10

# Loading Data
Please download the UCI datset from the above link and use your favorite data loading methods to set up (`x_train`, `y_train`) and (`x_val`, `y_val`) numpy arrays.

### Data Preparation

[Typical RNN models](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb) by convention, use a 3 dimensional tensor for the input data. This tensor is of shape `[number of examples, number of time steps, number of features]`. To incorporate the notion of *bags* and *sub-instances*, we extend this by adding an additional fourth dimension, thus making our input data shape - `[number of bags, number of sub-instances, number of time steps, number of features]`. Additionally, the typical shape of the one-hot encoded label tensor - `[number of examples, number of outputs]` is extended to incorporate sub-instance level labels, thus making it `[number of bags, number of sub-instances, number of output classes]`.

Specifically for HAR dataset, the data creation algorithm looks something like this.

```
def createData(X, Y, subinstanceWidth, subinstanceStride):
    '''
    TODO: Provide actual code
    
    Here X and Y are time series input from HAR and their labels. This methods
    chops the sequences into temporarily ordered set of sub-instances. All 
    sub-instances are given the same label as the bag.
    '''
    assert len(X) == len(Y)
    assert len(X.shape) == 3
    assert len(Y.shape) == 2
    
    X_out = []
    Y_out = []
    
    for i in range(len(X)):
        bag = X[i]
        bagLabel = Y[i]
        
        instances = breakBagIntoInstances(bag, subinstanceWidth, subinstaceStride)
        instanceLabels = [Y[i]] * len(instances)
        X_out.append(instances)
        Y_out.append(instanceLabels)
```

In [3]:
# Loading the data and shit
x_train, y_train = np.load('./HAR/48_16/x_train.npy'), np.load('./HAR/48_16/y_train.npy')
x_test, y_test = np.load('./HAR/48_16/x_test.npy'), np.load('./HAR/48_16/y_test.npy')
x_val, y_val = np.load('./HAR/48_16/x_val.npy'), np.load('./HAR/48_16/y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (6220, 6, 48, 9)
y_train shape is: (6220, 6, 6)
x_test shape is: (1132, 6, 48, 9)
y_test shape is: (1132, 6, 6)


# Computation Graph

The *EMI-RNN* computation graph consists of three parts:
1. `EMI_DataPipeline`: A data input pipeline that uses the Tensorflow Dataset API. This helps us implement an efficient data input pipeline.
2. `EMI_RNN`: An implementation of the abstract `EMI_RNN` class, for instance, `EMI_LSTM`,  which defines the forward computation or inference graph, and
3. `EMI_Trainer`: An instance of `EMI_Trainer` class which defines the loss functions and the training graph.

To build the computation graph, we create an instance of all the above and then connect them together.

The `EMI_BasicLSTM` class is an implementation that uses an LSTM cell and provides the LSTM output at each step for a secondary classifier to use. This secondary classifier is not implemented as part of `EMI_BasicLSTM` and is left to the user to define. The secondary classifier is define by overriding the `createExtendedGraph` method, and the `restoreExtendedgraph` method.

For the purpose of this example, lets use a simple linear secondary classifier.

In [4]:
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
EMI_BasicLSTM._createExtendedGraph = createExtendedGraph
EMI_BasicLSTM._restoreExtendedGraph = restoreExtendedGraph

In [5]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiLSTM = EMI_BasicLSTM(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS,
                        forgetBias=FORGET_BIAS, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

Now that we have all the elementary parts of the computation graph setup, we connect them together to form the forward graph.

In [6]:
# ... for good measure
np.random.seed(42)
tf.reset_default_graph()
g1 = tf.Graph()    
with g1.as_default():
    x_batch, y_batch = inputPipeline()
    y_cap = emiLSTM(x_batch)
    emiTrainer(y_cap, y_batch)

# EMI Driver

The `EMI_Driver` implements the `EMI_RNN` algorithm. TODO: Explain 



In [ ]:
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiLSTM, emiTrainer)

emiDriver.initializeSession(g1)
smsOut = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train, y_train=y_train, bag_train=BAG_TRAIN, 
              x_val=x_val, y_val=y_val, bag_val=BAG_VAL, numIter=NUM_ITER,
              numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE, numEpochs=NUM_EPOCHS)

Round: 0
Epoch   4 Batch   180 (  960) Loss 0.00487 Acc 0.88021 | Val acc 0.92192 | Model saved to /tmp/model, global_step 1000
Epoch   4 Batch   180 (  960) Loss 0.00384 Acc 0.89583 | Val acc 0.93176 | Model saved to /tmp/model, global_step 1001
Epoch   4 Batch   180 (  960) Loss 0.00352 Acc 0.90104 | Val acc 0.93326 | Model saved to /tmp/model, global_step 1002
INFO:tensorflow:Restoring parameters from /tmp/model-1002
Round: 1
Epoch   4 Batch   180 (  960) Loss 0.00369 Acc 0.89583 | Val acc 0.93480 | Model saved to /tmp/model, global_step 1003
Epoch   4 Batch   180 (  960) Loss 0.00323 Acc 0.90104 | Val acc 0.93441 | Model saved to /tmp/model, global_step 1004
Epoch   4 Batch   180 (  960) Loss 0.00289 Acc 0.90104 | Val acc 0.93475 | Model saved to /tmp/model, global_step 1005
INFO:tensorflow:Restoring parameters from /tmp/model-1003
Round: 2
Epoch   4 Batch   180 (  960) Loss 0.00323 Acc 0.90104 | Val acc 0.93441 | Model saved to /tmp/model, global_step 1006
Epoch   4 Batch   180 ( 

In [ ]:
opList = emiTrainer.accTilda
acc = emiDriver.runOps(opList, x_test, y_test, BATCH_SIZE)
print("Final step test accuracy: %f" % np.mean(acc))